In [1]:
# Imports
from main.models import Neighborhood
from django.contrib.gis.geos import (
    GEOSGeometry, 
    Polygon,
    Point,
    MultiPolygon,
)
from pprint import pprint

In [23]:
# Find the neighborhood that contains a given point
test_point = Point(-118.41303, 34.0658684) # Peninsula Hotel
print(Neighborhood.objects.filter(mpoly__contains=test_point))

# Check if a particular neighborhood contains a point
beverly_hills = Neighborhood.objects.get(name="Beverly Hills")
print(beverly_hills.mpoly.contains(test_point))

<QuerySet [<Neighborhood: Beverly Hills>]>
True
